In [1]:
# Ngoc Nguyen

from bs4 import BeautifulSoup
import requests
import lxml
import sqlite3
import time

time_start = time.time()

conn = sqlite3.connect('Collectdata.db')
c = conn.cursor()
try:
    conn.execute("DROP TABLE EVENTS")
    conn.execute("DROP TABLE BIRTHS")
    conn.execute("DROP TABLE DEATHS")
    conn.execute("DROP TABLE HOLIDAYS")
except:
    print('No table')

c.execute("CREATE TABLE EVENTS (DAY INT NOT NULL, MONTH TEXT NOT NULL, YEAR TEXT, DESCRIPTION TEXT)")
c.execute("CREATE TABLE BIRTHS (DAY INT NOT NULL, MONTH TEXT NOT NULL, YEAR TEXT, DESCRIPTION TEXT)")
c.execute("CREATE TABLE DEATHS (DAY INT NOT NULL, MONTH TEXT NOT NULL, YEAR TEXT, DESCRIPTION TEXT)")
c.execute("CREATE TABLE HOLIDAYS (DAY INT NOT NULL, MONTH TEXT NOT NULL, DESCRIPTION TEXT)")

conn.commit()


months =['January', 'February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Define a function to extract data from each day

def extract_data(month, day):
    url_addr = 'https://en.wikipedia.org/wiki/' + month + '_' + str(day)
    file = requests.get(url_addr).text

    file_html = BeautifulSoup(file, 'html.parser')

    # First extract information from Events

    file_events = file_html.find('span', id='Events')
    event_file_all = file_events.find_all_next(['li', 'span'])

    for event in event_file_all:
        if (event.get('id') != 'Births'):
            if (event.name == 'li'):
                index1 = (str(event.text)).find('–')
                index2 = (str(event.text)).find('‐')
                if (index1==-1):
                    index = index2
                else:
                    index = index1
                year = (str(event.text))[:index].strip()
                description = (str(event.text))[index+1:].strip()
                c.execute("INSERT INTO EVENTS VALUES(?,?,?,?)", (day, month, year, description))
        else:
            break

    # Second extract information from Births
    file_births = file_html.find('span', id='Births')
    births_file_all = file_births.find_all_next(['li', 'span'])

    for birth in births_file_all:
        if birth.get('id')!='Deaths':
            if birth.name == 'li':
                index1 = str(birth.text).find('–')
                index2 = (str(birth.text)).find('‐')
                if (index1 == -1):
                    index = index2
                else:
                    index = index1
                year = (str(birth.text))[:index].strip()
                description = (str(birth.text))[index + 1:].strip()
                c.execute("INSERT INTO BIRTHS VALUES(?,?,?,?)", (day, month, year, description))
        else:
            break

    # Third extract information from Deaths
    file_deaths = file_html.find('span', id='Deaths')
    deaths_file_all = file_deaths.find_all_next(['li', 'span'])

    for death in deaths_file_all:
        if (death.get('id') != 'Holidays_and_observances'):
            if (death.name == 'li'):
                index1 = (str(death.text)).find('–')
                index2 = (str(death.text)).find('‐')
                if (index1 == -1):
                   index = index2
                else:
                   index = index1
                year = (str(death.text))[:index].strip()
                description = (str(death.text))[index + 1:].strip()
                c.execute("INSERT INTO DEATHS VALUES(?,?,?,?)", (day, month, year, description))
        else:
            break

    # Last extract information for Holidays
    file_holidays = file_html.find('span', id='Holidays_and_observances')
    holidays_file_all = file_holidays.find_all_next(['li', 'span'])

    for holiday in holidays_file_all:
        if (holiday.get('id') != 'References'):
            if holiday.name=='li':
                description = holiday.text
                c.execute("INSERT INTO HOLIDAYS VALUES(?,?,?)", (day, month, description))
        else:
            break

#Loop over month and day to get all information
for month in range(1, 13):
    time_start = time.time()
    for day in range(1, days[month-1]+1):
        extract_data(months[month-1], day)
    time_end = time.time()
    print('Finish month', months[month-1], ' in ', time_end-time_start)


#Commit and close the connection

conn.commit()
conn.close()

No table
Finish month January  in  28.89410138130188
Finish month February  in  22.073606967926025
Finish month March  in  32.33692002296448
Finish month April  in  28.50624942779541
Finish month May  in  36.09915041923523
Finish month June  in  25.07924485206604
Finish month July  in  23.58283042907715
Finish month August  in  32.86137866973877
Finish month September  in  28.003759622573853
Finish month October  in  27.895984649658203
Finish month November  in  27.42598819732666
Finish month December  in  26.946940660476685
